In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mitsui-commodity-prediction-challenge/target_pairs.csv
/kaggle/input/mitsui-commodity-prediction-challenge/train_labels.csv
/kaggle/input/mitsui-commodity-prediction-challenge/train.csv
/kaggle/input/mitsui-commodity-prediction-challenge/test.csv
/kaggle/input/mitsui-commodity-prediction-challenge/lagged_test_labels/test_labels_lag_1.csv
/kaggle/input/mitsui-commodity-prediction-challenge/lagged_test_labels/test_labels_lag_4.csv
/kaggle/input/mitsui-commodity-prediction-challenge/lagged_test_labels/test_labels_lag_3.csv
/kaggle/input/mitsui-commodity-prediction-challenge/lagged_test_labels/test_labels_lag_2.csv
/kaggle/input/mitsui-commodity-prediction-challenge/kaggle_evaluation/mitsui_inference_server.py
/kaggle/input/mitsui-commodity-prediction-challenge/kaggle_evaluation/mitsui_gateway.py
/kaggle/input/mitsui-commodity-prediction-challenge/kaggle_evaluation/__init__.py
/kaggle/input/mitsui-commodity-prediction-challenge/kaggle_evaluation/core/templates.py
/kaggle/inpu

In [2]:
import os
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.preprocessing import StandardScaler
import warnings
from tqdm import tqdm
import polars as pl
import logging
import time
import joblib

In [3]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

In [4]:
warnings.filterwarnings("ignore")

In [5]:
target_cols = [f'target_{i}' for i in range(424)]

In [6]:
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'learning_rate': 0.05,
    'num_leaves': 31,
    'n_estimators': 30,
    'early_stopping_rounds': 10,
    'verbose': -1,
    'random_state': 42,
    'n_jobs': -1,
    'device': 'gpu',
    'gpu_platform_id': 0,
    'gpu_device_id': 0
}

In [7]:
class Predictor:
    def __init__(self):
        logger.info("Initializing Predictor")
        start_time = time.time()
        self.input_dir = '/kaggle/input/mitsui-commodity-prediction-challenge/'
        self.output_dir = '/kaggle/working/'
        
        try:
            logger.info("Loading input files")
            self.train = pd.read_csv(os.path.join(self.input_dir, 'train.csv'))
            self.train_labels = pd.read_csv(os.path.join(self.input_dir, 'train_labels.csv'))
            self.target_pairs = pd.read_csv(os.path.join(self.input_dir, 'target_pairs.csv'))
        except Exception as e:
            logger.error(f"Failed to load input files: {e}")
            raise

        logger.info("Processing target pairs")
        self.target_pairs['pair'] = self.target_pairs['pair'].fillna('')
        self.target_pairs['asset1'] = self.target_pairs['pair'].apply(lambda x: x.split(' - ')[0] if ' - ' in x else x)
        self.target_pairs['asset2'] = self.target_pairs['pair'].apply(lambda x: x.split(' - ')[1] if ' - ' in x else np.nan)
        self.assets = set(self.target_pairs['asset1'].unique()) | set(self.target_pairs['asset2'].dropna().unique())
        
        logger.info("Adding features to training data")
        self.history = self.add_features(self.train.copy())
        self.history_labels = self.train_labels.copy()
        
        self.submissions = []
        self.scaler = StandardScaler()
        
        model_path = os.path.join(self.output_dir, 'models.pkl')
        if os.path.exists(model_path):
            logger.info("Loading pre-trained models")
            self.models = joblib.load(model_path)
        else:
            logger.info("Training models")
            self.models = self.train_models()
            logger.info("Saving pre-trained models")
            joblib.dump(self.models, model_path)
        
        logger.info("Caching scaled training features")
        df = pd.merge(self.history, self.history_labels, on='date_id', how='inner')
        X = df.drop(columns=['date_id'] + target_cols, errors='ignore')
        X = X.replace([np.inf, -np.inf], np.nan).fillna(0)
        self.X_scaled = pd.DataFrame(self.scaler.fit_transform(X), columns=X.columns)
        
        logger.info(f"Predictor initialization completed in {time.time() - start_time:.2f} seconds")

    def add_features(self, df, is_test=False):
        logger.info(f"Adding features to {'test' if is_test else 'training'} DataFrame")
        start_time = time.time()
        try:
            df['dow'] = df['date_id'] % 7
            df['month'] = df['date_id'] // 30 % 12
            
            for idx, row in self.target_pairs.iterrows():
                asset1, asset2, lag = row['asset1'], row['asset2'], row['lag']
                if asset1 in df.columns:
                    price1 = df[asset1].replace(0, np.nan)
                    df[f'{asset1}_ret1'] = np.log(price1 / price1.shift(1)).replace([np.inf, -np.inf], np.nan).fillna(0)
                    df[f'{asset1}_ret{lag}'] = np.log(price1 / price1.shift(lag)).replace([np.inf, -np.inf], np.nan).fillna(0)
                    for w in [5, 10]:
                        df[f'{asset1}_ma{w}'] = df[f'{asset1}_ret1'].rolling(w).mean().fillna(0)
                        df[f'{asset1}_std{w}'] = df[f'{asset1}_ret1'].rolling(w).std().fillna(0)
                
                if pd.notna(asset2) and asset2 in df.columns:
                    price2 = df[asset2].replace(0, np.nan)
                    df[f'{asset2}_ret1'] = np.log(price2 / price2.shift(1)).replace([np.inf, -np.inf], np.nan).fillna(0)
                    df[f'{asset2}_ret{lag}'] = np.log(price2 / price2.shift(lag)).replace([np.inf, -np.inf], np.nan).fillna(0)
                    for w in [5, 10]:
                        df[f'{asset2}_ma{w}'] = df[f'{asset2}_ret1'].rolling(w).mean().fillna(0)
                        df[f'{asset2}_std{w}'] = df[f'{asset2}_ret1'].rolling(w).std().fillna(0)
                    
                    if asset1 in df.columns and asset2 in df.columns:
                        df[f'spread_{asset1}_{asset2}'] = np.log(price1 / price2).replace([np.inf, -np.inf], np.nan).fillna(0)
                        df[f'spread_ret1_{asset1}_{asset2}'] = df[f'{asset1}_ret1'] - df[f'{asset2}_ret1']
            
            price_cols = [col for col in df.columns if any(k in col for k in ['Close', 'adj_close', 'Open', 'settlement_price'])]
            if price_cols:
                df['global_avg_ret'] = df[[f'{c}_ret1' for c in price_cols if f'{c}_ret1' in df.columns]].mean(axis=1).fillna(0)
                df['global_std_ret'] = df[[f'{c}_ret1' for c in price_cols if f'{c}_ret1' in df.columns]].std(axis=1).fillna(0)
            
            df = df.replace([np.inf, -np.inf], np.nan).fillna(0)
            logger.info(f"Feature engineering completed in {time.time() - start_time:.2f} seconds")
            return df
        except Exception as e:
            logger.error(f"Failed to add features: {e}")
            raise

    def train_models(self):
        logger.info("Merging history and labels for training")
        try:
            df = pd.merge(self.history, self.history_labels, on='date_id', how='inner')
            
            X = df.drop(columns=['date_id'] + target_cols, errors='ignore')
            y = df[target_cols].fillna(0)
            
            X = X.replace([np.inf, -np.inf], np.nan).fillna(0)
            
            logger.info("Scaling features")
            X_scaled = pd.DataFrame(self.scaler.fit_transform(X), columns=X.columns)
            
            train_size = int(0.8 * len(X_scaled))
            X_train, X_val = X_scaled[:train_size], X_scaled[train_size:]
            y_train, y_val = y[:train_size], y[train_size:]
            
            models = {}
            for col in tqdm(target_cols, desc="Training models"):
                try:
                    model = lgb.LGBMRegressor(**params)
                    model.fit(
                        X_train, y_train[col],
                        eval_set=[(X_val, y_val[col])],
                        eval_metric='rmse',
                        callbacks=[lgb.early_stopping(10, verbose=False)]
                    )
                    models[col] = model
                except Exception as e:
                    logger.error(f"Failed to train model for {col}: {e}")
                    raise
            return models
        except Exception as e:
            logger.error(f"Failed to train models: {e}")
            raise

    def get_lagged_labels(self, label_lags_1_batch, label_lags_2_batch, label_lags_3_batch, label_lags_4_batch):
        logger.info("Processing lagged labels")
        start_time = time.time()
        try:
            lagged_dfs = [df for df in [label_lags_1_batch, label_lags_2_batch, label_lags_3_batch, label_lags_4_batch] if not df.is_empty()]
            if not lagged_dfs:
                logger.info("No lagged labels provided")
                return pd.DataFrame()
            
            for i, df in enumerate(lagged_dfs, 1):
                logger.info(f"Lagged DataFrame {i} columns: {df.columns}")
            
            expected_cols = ['date_id'] + target_cols
            aligned_dfs = []
            for df in lagged_dfs:
                if 'release_date_id' in df.columns:
                    df = df.rename({'release_date_id': 'date_id'})
                
                df_cols = set(df.columns)
                missing_cols = set(expected_cols) - df_cols
                extra_cols = df_cols - set(expected_cols)
                
                logger.info(f"Missing columns in DataFrame: {missing_cols}")
                logger.info(f"Extra columns in DataFrame: {extra_cols}")
                
                if extra_cols:
                    df = df.drop(list(extra_cols))
                
                for col in missing_cols:
                    df = df.with_columns(pl.lit(0.0).alias(col))
                
                df = df.select(expected_cols)
                aligned_dfs.append(df)
            
            if aligned_dfs:
                lagged_df = pl.concat(aligned_dfs, how='vertical')
                lagged_df = lagged_df.sort('date_id').unique(subset=['date_id'])
                logger.info(f"Final lagged DataFrame columns: {lagged_df.columns}")
            else:
                lagged_df = pl.DataFrame(columns=expected_cols)
            
            logger.info(f"Lagged labels processing completed in {time.time() - start_time:.2f} seconds")
            return lagged_df.to_pandas()
        except Exception as e:
            logger.error(f"Failed to process lagged labels: {e}")
            raise

    def predict(self, test: pl.DataFrame, label_lags_1_batch: pl.DataFrame, label_lags_2_batch: pl.DataFrame, 
                label_lags_3_batch: pl.DataFrame, label_lags_4_batch: pl.DataFrame) -> pl.DataFrame:
        logger.info("Starting prediction")
        start_time = time.time()
        try:
            test_df = test.to_pandas()
            test_df = test_df.drop(['is_scored'], axis=1, errors='ignore')
            
            logger.info("Adding features to test data")
            test_df = self.add_features(test_df, is_test=True)
            
            new_labels = self.get_lagged_labels(label_lags_1_batch, label_lags_2_batch, label_lags_3_batch, label_lags_4_batch)
            if not new_labels.empty:
                logger.info("New lagged labels received, updating history_labels")
                self.history_labels = pd.concat([self.history_labels, new_labels], ignore_index=True)
                self.history_labels = self.history_labels.sort_values('date_id').drop_duplicates('date_id')
            
            current_date = test_df['date_id'].iloc[0]
            X_test = test_df[test_df['date_id'] == current_date].drop(columns=['date_id'], errors='ignore')
            X_test = X_test.replace([np.inf, -np.inf], np.nan).fillna(0)
            
            logger.info("Scaling test features")
            X_test_scaled = pd.DataFrame(self.scaler.transform(X_test), columns=X_test.columns)
            
            logger.info("Making predictions")
            preds = {col: model.predict(X_test_scaled)[0] for col, model in self.models.items()}
            
            sub_for_save = pl.DataFrame({'date_id': [current_date], **preds})
            self.submissions.append(sub_for_save.to_pandas())
            
            logger.info("Saving submission to parquet")
            submission_df = pd.concat(self.submissions, ignore_index=True)
            submission_df.to_parquet(os.path.join(self.output_dir, 'submission.parquet'), index=False)
            
            sub = pl.DataFrame(preds)
            logger.info(f"Prediction DataFrame columns: {sub.columns}")
            logger.info(f"Prediction completed in {time.time() - start_time:.2f} seconds")
            return sub
        except Exception as e:
            logger.error(f"Prediction failed: {e}")
            raise

In [8]:
if __name__ == "__main__":
    from kaggle_evaluation.mitsui_inference_server import MitsuiInferenceServer
    logger.info("Starting main execution")
    try:
        predictor = Predictor()
        inference_server = MitsuiInferenceServer(predictor.predict)
        if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
            logger.info("Running inference server in competition mode")
            inference_server.serve()
        else:
            logger.info("Running local gateway")
            inference_server.run_local_gateway(('/kaggle/input/mitsui-commodity-prediction-challenge/',))
    except Exception as e:
        logger.error(f"Failed to start inference server: {e}")
        raise

Training models:   0%|          | 0/424 [00:00<?, ?it/s]1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
Training models: 100%|██████████| 424/424 [14:01<00:00,  1.99s/it]
